import numpy as np
# ^^^ pyforest auto-imports - don't write above this line


# Main Question: Do American teams have more American Players than Canadian Teams and vice versa for 2018-2019? 

## Imports

In [1]:
from functions import *
from scipy.stats import chi2_contingency


Bad key "text.kerning_factor" on line 4 in
/Users/gregfeliu/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


### Data

In [2]:
player_df = pd.read_csv("player_df", index_col=0)
groupby_df_sum = player_df.groupby('Team_id').sum()
groupby_df_sum.reset_index(inplace=True)

# Calculating Percent of Players in each Group

### Splitting by Nation of Team

In [3]:
can_teams = player_df[player_df['Canadian_team'] == True]
can_team_ids = list(can_teams.Team_id.unique())

In [4]:
# making separate dataframes
canadian_teams_groupby = groupby_df_sum[groupby_df_sum['Team_id'].isin(can_team_ids)]
amer_teams_groupby = groupby_df_sum[~groupby_df_sum['Team_id'].isin(can_team_ids)]

In [5]:
all_players_in_can_teams = canadian_teams_groupby.Dummy.sum()
all_can_players_in_can_teams = int(canadian_teams_groupby.Canadian_player.sum())
all_amer_players_in_can_teams = int(canadian_teams_groupby.Amer_player.sum())

In [6]:
all_players_in_amer_teams = amer_teams_groupby.Dummy.sum()
all_can_players_in_amer_teams = int(amer_teams_groupby.Canadian_player.sum())
all_amer_players_in_amer_teams = int(amer_teams_groupby.Amer_player.sum())

#### Calculating percentage of canadians in each group

In [7]:
canadians_in_canada_teams_pct = all_can_players_in_can_teams / all_players_in_can_teams
canadians_in_canada_teams_pct

0.5

In [8]:
canadians_in_amer_teams_pct = all_can_players_in_amer_teams / all_players_in_amer_teams
canadians_in_amer_teams_pct

0.4343891402714932

#### Calculating percentage of americans in each group

In [9]:
americans_in_canada_teams_pct = all_amer_players_in_can_teams / all_players_in_can_teams
americans_in_canada_teams_pct

0.24193548387096775

In [10]:
# slightly more americans in american teams than in canadian teams
americans_in_amer_teams_pct = all_amer_players_in_amer_teams / all_players_in_amer_teams
americans_in_amer_teams_pct

0.2895927601809955

# Testing for Statistical Significance
test for significance (5%)

## Making Contingency Table 

In [11]:
# canadian teams
other_nats_in_can_teams = int(all_players_in_can_teams - (all_can_players_in_can_teams + 
                                                      all_amer_players_in_can_teams))

In [12]:
# american teams
other_nats_in_amer_teams = int(all_players_in_amer_teams - (all_can_players_in_amer_teams + 
                                                      all_amer_players_in_amer_teams))

In [13]:
#               canadians | americans | other
#  canadian teams
# american  teams
contingency_table = np.array([[all_can_players_in_can_teams, 
                              all_amer_players_in_can_teams, 
                              other_nats_in_can_teams],
                            [all_can_players_in_amer_teams,
                            all_amer_players_in_amer_teams,
                            other_nats_in_amer_teams
                            ]])

<IPython.core.display.Javascript object>

## Running Test

In [14]:
chi2, p_value, deg_of_freedom, expected = chi2_contingency(contingency_table)
p_value

In [16]:
contingency_table

array([[31, 15, 16],
       [96, 64, 61]])

In [17]:
expected  

array([[27.82332155, 17.30742049, 16.86925795],
       [99.17667845, 61.69257951, 60.13074205]])

## TEMPORARY CONCLUSION

There is NOT a significant difference in terms of the numbers of canadians, americans, and other nationalities between American and Canadian NHL Teams. 

# Testing by for players playing in home country

The previous test found that between teams in the two countries, there is no difference in terms of the number of canadians, americans, and other nationalities playing in the two countries. Here, I want to ask a more specific question, are there more Canadians playing in canada and Americans playing in America? 

## Canadians playing in canada

### Making contingency table

In [18]:
non_canadians_in_can_teams = all_amer_players_in_can_teams + other_nats_in_can_teams
non_canadians_in_amer_teams = all_amer_players_in_amer_teams + other_nats_in_amer_teams

In [19]:
can_contingency_table = np.array([[all_can_players_in_can_teams, non_canadians_in_can_teams],
                                 [all_can_players_in_amer_teams, non_canadians_in_amer_teams]])

<IPython.core.display.Javascript object>

## Running Test

In [20]:
can_chi2, can_p_value, can_deg_of_freedom, can_expected = chi2_contingency(can_contingency_table)
can_p_value

0.43926931071597886

## Americans Playing in America

In [21]:
non_americans_in_can_teams = all_can_players_in_can_teams + other_nats_in_can_teams
non_americans_in_amer_teams = all_can_players_in_amer_teams + other_nats_in_amer_teams

In [22]:
amer_contingency_table = np.array([[all_amer_players_in_can_teams, non_americans_in_can_teams],
                                 [all_amer_players_in_amer_teams, non_americans_in_amer_teams]])

<IPython.core.display.Javascript object>

## Running Test

In [23]:
amer_chi2, amer_p_value, amer_deg_of_freedom, amer_expected = chi2_contingency(amer_contingency_table)
amer_p_value

## TEMPORARY CONCLUSION 

There are more canadians in canadian teams, but not more americans in american teams. In other words, canadians stay local more than americans do. 

# Conclusion

After all of these tests, we can say that there is no significant difference between the U.S. and Canadian teams in the nationalities of its players. In no case did the p_value for the categories come close to statistical significance. 

This is surprising for a few reasons:

**taxes**
 All players pay national taxes and some have to pay state/province taxes on top of that amount. This almost always means that Canadian player pay much more in taxes than Americans do (though there is lots of variation in American state taxes on salaries) (https://www.reddit.com/r/winnipegjets/comments/45wn74/are_canadian_teams_players_like_the_jets_paid_in/).

**weather**
Although ice hockey is a winter sport, if one has the choice between south Florida or Winnipeg, which location would you rather work/live 8-10 months of the year in? I know which one I would pick...